### Tarea 1 (10 puntos). Creación y operaciones básicas con PySpark

- Instalar en un entorno local o ejecutar Spark en algún servidor en línea (como Google Colab)
- Elegir un conjunto de datos para trabajar durante el tetramestre, definirlo y explicar por qué se elige
- Cargar el conjunto de datos mediante PySpark
- Usar PySpark para filtrar datos, generar estadísticas descriptivas básicas y realizar algunas operaciones aritméticas entre registros y columnas
- Crear un repositorio público para el curso y publicar en un cuaderno esta primera tarea

In [1]:
import os
import sys

import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession.builder.appName("Mi primera tarea").getOrCreate()

In [2]:
df = spark.read.csv("mortgage_data.csv", header=True, inferSchema=True)

# Mostrar las primeras filas del DataFrame
df.show(5)

+------+---------+--------+--------+-----+-------------+--------------------+------+----+-------+-----+----------+------+--------+
|SOURCE|FREQUENCY|SERIESID|GEOLEVEL|GEOID|      GEONAME|              MARKET|PERIOD|YEAR|QUARTER|MONTH|SUPPRESSED|VALUE1|  VALUE2|
+------+---------+--------+--------+-----+-------------+--------------------+------+----+-------+-----+----------+------+--------+
|  NMDB|Quarterly|TOT_ORIG|National|  USA|United States|       All Mortgages|1998Q1|1998|      1|    3|         0|2544.0|277368.0|
|  NMDB|Quarterly|TOT_ORIG|National|  USA|United States|All Mortgages (Ho...|1998Q1|1998|      1|    3|         0|1044.0|108582.0|
|  NMDB|Quarterly|TOT_ORIG|National|  USA|United States|All Mortgages (Re...|1998Q1|1998|      1|    3|         0|1499.0|168786.0|
|  NMDB|Quarterly|TOT_ORIG|National|  USA|United States|       All Mortgages|1998Q2|1998|      2|    6|         0|2910.0|319970.0|
|  NMDB|Quarterly|TOT_ORIG|National|  USA|United States|All Mortgages (Ho...|1998Q2

# Conjunto de Datos: National Mortgage Database (NMDB®)

## Descripción del Conjunto de Datos
El **National Mortgage Database (NMDB®)** es una muestra representativa a nivel nacional del 5% de las hipotecas residenciales en los Estados Unidos. Este conjunto de datos se publica para cumplir con los requisitos establecidos por la sección 1324(c) de la Federal Housing Enterprises Financial Safety and Soundness Act de 1992, que fue enmendada por la Housing and Economic Recovery Act de 2008. La base de datos recopila características detalladas de las hipotecas, tanto de las empresas como de no empresas, y busca hacer esta información accesible al público protegiendo la privacidad de los prestatarios.

## Motivo para Elegir Este Conjunto de Datos

### 1. Relevancia en la Industria Financiera
El NMDB proporciona una visión integral del mercado hipotecario en los EE. UU., lo cual es fundamental para entender las dinámicas financieras y económicas. Este tipo de datos es extremadamente útil para realizar análisis de riesgos, comportamiento del consumidor y predicciones de mercado.

### 2. Complejidad y Variedad de Datos
La base de datos contiene múltiples variables que van desde características geográficas y temporales hasta detalles específicos de las hipotecas, como valores y tipos de préstamo. Esta variedad permite realizar análisis multivariados y aplicar técnicas avanzadas de Big Data y Machine Learning.

### 3. Volumen de Datos
Al tratarse de una muestra del 5% de todas las hipotecas residenciales, el volumen es significativo, con millones de registros. Este tamaño es ideal para practicar y aplicar técnicas de procesamiento distribuido y análisis masivo de datos, que son componentes clave en Big Data.

### 4. Impacto Social y Económico
El análisis de este conjunto de datos puede contribuir a una mejor comprensión del mercado hipotecario, lo que tiene un impacto directo en la economía de los hogares y la estabilidad financiera de la nación.

### 5. Beneficio Personal y Profesional
Trabajar con este conjunto de datos tiene un valor añadido personal, ya que estoy empleado en una empresa que se dedica a facilitar hipotecas. Analizar y entender los patrones y tendencias del mercado hipotecario a través del NMDB me permitirá aplicar este conocimiento directamente en mi trabajo, mejorando la toma de decisiones y el desarrollo de productos en mi entorno profesional.

---

En resumen, el NMDB es un conjunto de datos robusto y relevante para aplicar técnicas de Big Data y análisis avanzado, y puede proporcionar un excelente marco para el desarrollo de proyectos complejos a lo largo del tetramestre.


# Definición del Conjunto de Datos: NMDB® Aggregate Statistics

El conjunto de datos **NMDB® Aggregate Statistics** ofrece una visión general de las estadísticas hipotecarias agregadas en los Estados Unidos. A continuación, se presenta la descripción detallada de cada campo:

| **Campo**       | **Descripción**                                                                                       | **Tipo**     | **Ejemplos/Valores**                                            |
|-----------------|-------------------------------------------------------------------------------------------------------|-------------|----------------------------------------------------------------|
| **SOURCE**      | Fuente de los datos.                                                                                  | String      | `NMDB`                                                         |
| **FREQUENCY**   | Frecuencia de la serie de datos.                                                                      | String      | `Annual`, `Quarterly`, `Monthly`                               |
| **SERIESID**    | Identificador del campo estadístico.                                                                  | String      | `New Residential Mortgages`, `Outstanding Mortgages`           |
| **GEOLEVEL**    | Nivel de geografía.                                                                                   | String      | `National`, `State`, `Metro Area`                              |
| **GEOID**       | Identificador geográfico.                                                                             | String      | `USA`, `FL`, `23420`                                           |
| **GEONAME**     | Nombre de la geografía.                                                                               | String      | `United States`, `Florida`, `Fresno, CA`                       |
| **MARKET**      | Mercado o submercado hipotecario.                                                                     | String      | `All Mortgages`, `Enterprise Acquisitions`, `Government`       |
| **PERIOD**      | Período de tiempo, incluyendo año, trimestre o mes.                                                   | String      | `2009`, `2009Q2`, `200906`                                     |
| **YEAR**        | Año del dato.                                                                                         | Numeric     | `2009`                                                         |
| **QUARTER**     | Trimestre del año.                                                                                    | Numeric     | `1`, `2`, `3`, `4`                                             |
| **MONTH**       | Mes del dato. Incluye ceros a la izquierda en caso necesario.                                         | Numeric     | `01`, `12`                                                     |
| **SUPPRESSED**  | Indicador de supresión de valor.                                                                      | Numeric     | `1` = Suprimido, `0` = No suprimido                            |
| **VALUE1**      | Valores ponderados por el número de préstamos.                                                        | Numeric     | Estadísticas/Agregados ponderados por el número de hipotecas   |
| **VALUE2**      | Valores ponderados por el volumen en dólares del préstamo.                                            | Numeric     | Estadísticas/Agregados ponderados por el valor del monto       |

## Ejemplos de Uso

- **SOURCE**: Indica el origen de los datos, siempre es "NMDB".
- **FREQUENCY**: Permite analizar los datos de manera anual, trimestral o mensual, lo cual es útil para realizar análisis temporales.
- **SERIESID**: Identifica el tipo de estadística que se está analizando, como hipotecas nuevas o el saldo pendiente de hipotecas.
- **GEOLEVEL** y **GEOID**: Determinan el nivel y la ubicación geográfica de los datos, permitiendo realizar análisis específicos por región, estado o área metropolitana.
- **MARKET**: Especifica el mercado hipotecario, como todos los préstamos hipotecarios o adquisiciones de empresas.
- **PERIOD**, **YEAR**, **QUARTER**, **MONTH**: Definen el marco temporal para el análisis de los datos.
- **SUPPRESSED**: Indica si los valores han sido suprimidos por razones de privacidad.
- **VALUE1** y **VALUE2**: Proporcionan datos agregados ponderados, ya sea por número de préstamos o por volumen de préstamo, lo cual es útil para análisis cuantitativos. 

Este conjunto de datos ofrece una base sólida para el análisis del mercado hipotecario en diferentes niveles geográficos y temporales, facilitando estudios de tendencias, comparaciones regionales y segmentación de mercado.


In [3]:
# Filtrar registros donde el año es 2000
df_filtered = df.filter(df.YEAR == 2000)

# Mostrar el resultado
df_filtered.show(10)


+------+---------+--------+--------+-----+-------------+--------------------+------+----+-------+-----+----------+------+--------+
|SOURCE|FREQUENCY|SERIESID|GEOLEVEL|GEOID|      GEONAME|              MARKET|PERIOD|YEAR|QUARTER|MONTH|SUPPRESSED|VALUE1|  VALUE2|
+------+---------+--------+--------+-----+-------------+--------------------+------+----+-------+-----+----------+------+--------+
|  NMDB|Quarterly|TOT_ORIG|National|  USA|United States|       All Mortgages|2000Q1|2000|      1|    3|         0|1630.0|184966.0|
|  NMDB|Quarterly|TOT_ORIG|National|  USA|United States|All Mortgages (Ho...|2000Q1|2000|      1|    3|         0|1038.0|126198.0|
|  NMDB|Quarterly|TOT_ORIG|National|  USA|United States|All Mortgages (Re...|2000Q1|2000|      1|    3|         0| 592.0| 58768.0|
|  NMDB|Quarterly|TOT_ORIG|National|  USA|United States|       All Mortgages|2000Q2|2000|      2|    6|         0|1945.0|230406.0|
|  NMDB|Quarterly|TOT_ORIG|National|  USA|United States|All Mortgages (Ho...|2000Q2

In [4]:

# Estadisticas descriptivas
df.select("PERIOD","YEAR","VALUE1", "VALUE2").describe().show()


+-------+-------+-----------------+------------------+-----------------+
|summary| PERIOD|             YEAR|            VALUE1|           VALUE2|
+-------+-------+-----------------+------------------+-----------------+
|  count|2007264|          2007264|           1910496|          1841280|
|   mean|   NULL|2010.378640776699| 53.38525728397144|776.8076042753173|
| stddev|   NULL|7.434004686891715|132.24802817927338| 11177.2213759521|
|    min| 1998Q1|             1998|               0.0|              0.0|
|    max| 2023Q3|             2023|            6161.0|        1159331.0|
+-------+-------+-----------------+------------------+-----------------+



In [5]:
from pyspark.sql.functions import col,mean

# Calcular el valor promedio de los préstamos (volumen en dólares / cantidad de hipotecas)
df_with_avg_loan = df.withColumn("Average_Loan_Value", col("VALUE2") / col("VALUE1"))

# Mostrar las primeras filas con la nueva columna
df_with_avg_loan.select("VALUE1", "VALUE2", "Average_Loan_Value").show(10)



+------+--------+------------------+
|VALUE1|  VALUE2|Average_Loan_Value|
+------+--------+------------------+
|2544.0|277368.0|109.02830188679245|
|1044.0|108582.0|104.00574712643679|
|1499.0|168786.0|112.59906604402936|
|2910.0|319970.0| 109.9553264604811|
|1482.0|157679.0|106.39608636977059|
|1427.0|162291.0|113.72880168185003|
|2817.0|317604.0| 112.7454739084132|
|1467.0|157723.0|107.51397409679618|
|1349.0|159881.0|118.51816160118607|
|3414.0|392280.0|114.90333919156414|
+------+--------+------------------+
only showing top 10 rows



In [6]:
# Agrupar por 'MARKET' y calcular el promedio de VALUE1 y VALUE2
grouped_df = df.groupBy("MARKET").agg(
    mean("VALUE1").alias("Avg_VALUE1"),
    mean("VALUE2").alias("Avg_VALUE2")
)
grouped_df.show()


+--------------------+------------------+------------------+
|              MARKET|        Avg_VALUE1|        Avg_VALUE2|
+--------------------+------------------+------------------+
|        Jumbo Market| 72.77505778604088| 371.7913516684048|
| Conventional Market|55.846408471936144|1751.1087721584993|
|Jumbo Market (Ref...| 73.97360811014525| 223.9177320125127|
|Conforming Market...| 51.96935581126575|1015.2194382168951|
|   Conforming Market|55.420640922566704|1735.9718287278415|
|Enterprise Acquis...| 50.97929375408289| 693.9394590719459|
|       All Mortgages| 56.92266720265309|2067.2581725755895|
|Other Conforming ...| 45.15587030802466| 190.2432755474448|
|All Mortgages (Ho...| 50.96085121350683|  908.667456986449|
|Conventional Conf...| 51.55158409125166| 885.6663347236697|
|All Mortgages (Re...| 53.25553741018047|1198.6087904066721|
|Conforming Market...|  49.6940430129139| 760.6912526068846|
|Conventional Mark...| 50.64957288578461| 722.1131373826917|
|Enterprise Acquis...| 4